In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size= 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [34]:
with open('WizardOfOz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [35]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [36]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [37]:
def get_batch(split):
    data = train_data if split =='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y= x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
#print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[54, 72, 56, 58, 67, 72, 62, 68],
        [72,  0, 76, 58, 71, 58,  1, 67],
        [58, 65, 69, 58, 57,  1, 66, 58],
        [73, 61, 62, 72,  9,  3,  1, 61]], device='cuda:0')
targets:
tensor([[72, 56, 58, 67, 72, 62, 68, 67],
        [ 0, 76, 58, 71, 58,  1, 67, 58],
        [65, 69, 58, 57,  1, 66, 58,  1],
        [61, 62, 72,  9,  3,  1, 61, 58]], device='cuda:0')


In [38]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [39]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


zX0p
8muKFNy9WLMA91cD]Z8n_M*IcbGJ﻿IN2[&﻿JUpLZsYGXnWE?GZ*wHfxZ﻿?GM(GRZBcLIdqg3whgtVb6P4:K't;ng&jy7G(k(kRQx.rFm.:zxk,(&5xGXOBIL3"zBH9z9!﻿c_*VJ5].﻿heEcKAV4a LKhHP4dqUJ3e-i"z9hy"R a]6foGXjVlNWi[GXkjLzSjWr'1-(kFH&Xi7qPHNibi'nWK;:)V)jGoPGspJx"C]jx("BGZE
l*Ii.Ye H"C63Z_xQ﻿ubK2﻿I;Y*w."5SM(;P]MFQLz7p!;5"1:.[Jwjec]PM54?OOP;ZN'eH&YRM291c_FE3XFr'.]aq5)
*IdGt0
Q:R&u"CR;n8sYCdCd'GsWVDr6,O9W
lBjxZ﻿ckJ1XOo_xk[vhtyg[SvPSdj'nA,xiqwa]IdgRywSt ushRMYrVh&--zJ71 u!xwzd&:tEMsBAlWkFM0X9Wo,_[z.0;'R9i[f(F!WkFO2HVBhiL3rvM


In [40]:
#crete a pytorch oprimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
        
    
    #sample a batch of data
    xb, yb = get_batch('train')
    #eval the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.824, val loss: 4.793
step: 250, train loss: 4.744, val loss: 4.764
step: 500, train loss: 4.663, val loss: 4.682
step: 750, train loss: 4.613, val loss: 4.631
4.610947132110596


In [41]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


s69LV'C8E7Lu.Wuf.(
Sp4!.3jQx"-]*1LBv_1(_JInnFIS?[R(BY_(F7dCkzv﻿g&2Nac*(]a2;REQp7'3!Q:VHf
.sdCC]kZ.zCE12;P?*a;ZrsKpI?(6atc89H0Oit wW-HZ_WYOPhmS(Iq.J_sJr jxQNJ:Cd'C'Sl_QXNN]8]DDnnnv'DnnnhtJ﻿cM(4﻿R;XlN0M*V-g81P36w0)OKBJTzWzU)E7P GXCFJwbjy9BuDDGvGKtF&e GE[&KK_JaWz6YIdeoP&X&JcpdS&e ,bFcNJ3:HVHnnncA3rfG6,YsRHVl_FQEc﻿QCk[)7P6CAW8RM(6tUE7&5,lW:A TUm3V98s,Ys7p2wK
?p;5WSwS7Y]0&b.J﻿3bz1krUYgWRPPPjaWVL[1"4E;TcNy&﻿?eIUpf3PmDrFdS[9R&Jatt,xZ2!K
-LB(kz!Z﻿.-8nFbk9X'A,0:)E]PNNu_o7CX0A)izV9L3-S;m_0,p0si﻿:C
Qd4igz2
